# Wave Watch III 
## Using erddap-python and xarray

WaveWatch III (WW3) is a third-generation wave model developed by the National Oceanic and Atmospheric Administration (NOAA) and other international research groups. It is a computer-based numerical model used to predict ocean wave conditions, including significant wave heights, wave periods, and wave directions for different regions around the world. WW3 provides wave forecasts for both deep water and coastal areas, which are important for various applications such as marine navigation, offshore industries, coastal management, and recreational activities.

WaveWatch III operates by simulating the growth, decay, and propagation of wind-generated waves in the ocean. The model takes into account factors such as wind speed, fetch (the distance over which the wind blows), and the duration for which the wind has been blowing. The model also accounts for wave interactions, swell propagation, and wave energy dissipation due to whitecapping and bottom friction. WaveWatch III is capable of producing global and regional wave forecasts, which are often used by weather forecast centers, oceanographers, and marine operators to make informed decisions regarding ocean activities.

### Global Wave Model
The Pacific Islands Ocean Observing System (PacIOOS) at the University of Hawaii, in collaboration with NOAA's National Centers for Environmental Prediction (NCEP) and the National Weather Service (NWS) Honolulu, has implemented a global-scale WaveWatch III (WW3) dataset. This dataset provides a 7-day wave model forecast with a 5-day hourly output at a resolution of approximately 50 km or 0.5 degrees. The WW3 model is forced by NOAA/NCEP's Global Forecast System (GFS) winds and is designed to capture large-scale ocean waves, while also providing spectral boundary conditions for Hawaii and other Pacific regional WW3 models.

The dataset, titled "WaveWatch III (WW3) Global Wave Model," is available online at the source URL: https://pae-paha.pacioos.hawaii.edu/erddap/griddap/ww3_global.html. It covers the geographical extent from 0.0 degrees Westernmost_Easting to -77.5 degrees Southernmost_Northing. The time coverage for this dataset starts from November 7, 2010, at 21:00:00 UTC and is updated continuously (The forecast is run daily and gets updated on (https://www.pacioos.hawaii.edu/waves/model-global/#about) around 2:00 AM Hawaiʻi Standard Time (UTC-10) every morning.). The time coverage resolution for this dataset is one hour.

While the dataset has been developed with significant care and effort, users are advised to exercise caution when using the data for any ocean-related activities due to the possibility of errors from various sources. More information and references for the WaveWatch III model can be found at https://polar.ncep.noaa.gov/waves/wavewatch/.

#### Access  
A list of access points for this data is here: https://www.pacioos.hawaii.edu/waves/model-global/#access

In this example we will show how to access the data via ERDDAP using errdap python `https://github.com/hmedrano/erddap-python`. ERDDAP is a data server that gives you a simple, consistent way to download subsets of gridded and tabular scientific datasets in common file formats and make graphs and maps. `erddap-python` is a python client for the ERDDAP Restful API, it can obtain server status metrics, provides search methods, gives tabledap and griddap class objects for metadata and data access.

**This is not an MHKiT module but a package MHKiT users may find useful to access WW3 and other data hosted on ERRDAP servers.**




https://github.com/ERDDAP
https://www.pacioos.hawaii.edu/waves/model-global/#access
https://pae-paha.pacioos.hawaii.edu/erddap/griddap/ww3_global.html

#### Step 1: Install erddap-python

```
pip install erddap-python
```


#### Step 2: Search for "WW3" on the ERDDAP server

You can search via a webbrowser (https://coastwatch.pfeg.noaa.gov/erddap/search/index.html?page=1&itemsPerPage=1000) or use `errdap-python` to search:

In [3]:
from erddapClient import ERDDAP_Server

remote = ERDDAP_Server('https://coastwatch.pfeg.noaa.gov/erddap')
try:
     searchResults = remote.search(searchFor="WW3")
     print("Search results:", searchResults)
     print("=====================================")

     ww3 = searchResults[0]
     print(f'DataSet ID: {ww3.datasetid}')
     print(f'Protocol: {ww3.protocol}')
     print(f'Dimensions: {list(ww3.dimensions.keys())}')
     print(f'Variables: {list(ww3.variables.keys())}')  
except: 
     print("No search results found. Wait a few minutes and try again.")
         


ERDDAP Error: ""
No search results found. Wait a few minutes and try again.


We get 1 search result. This gives us a single search result with the Dataset ID "NWW3_Global_Best" and is of of type (protocol) "Griddap". Using these two parameters we can access the data.

#### Step 3: Access the WW3 dataset and save it as a NetCDF file


In [2]:
import xarray as xr
from erddapClient import ERDDAP_Griddap

remote = ERDDAP_Griddap('https://coastwatch.pfeg.noaa.gov/erddap', 'NWW3_Global_Best')

ds = (
    remote.setResultVariables(['Tper']).setSubset(
        time=slice('2011-01-01','2011-01-02'),
        depth=0,
        latitude=slice(-77.5,77.5),
        longitude=slice(0, 359.5)
    )
    .getData(filetype='nc')
)

print("Writing to file" )
with open(path,'wb') as fb:
    fb.write(ds)

ds = xr.open_dataset('ww3.nc')

ERDDAP Error: ""


HTTPError: 503 Server Error: Service Unavailable for url: https://coastwatch.pfeg.noaa.gov/erddap/info/NWW3_Global_Best/index.json

#### Step 4: Plot the mean Tper
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 2.5))
mean_Tper = ds['Tper'].mean('time')
mean_Tper = mean_Tper.transpose('latitude', 'longitude')
mean_Tper = mean_Tper.isel(depth=0)
mean_Tper.plot.contourf(x='longitude', y='latitude')
plt.show()

In [23]:
print(f'Dimensions: {list(ww3.dimensions.keys())}')
print(f'Variables: {list(ww3.variables.keys())}')


Dimensions: ['time', 'depth', 'latitude', 'longitude']
Variables: ['Tdir', 'Tper', 'Thgt', 'sdir', 'sper', 'shgt', 'wdir', 'wper', 'whgt']


In [22]:
list(ww3.dimensions.keys())

['time', 'depth', 'latitude', 'longitude']

In [32]:
ww3.dimensions['depth'].data
ww3.dimensions['latitude'].data

Float64Index([-77.5, -77.0, -76.5, -76.0, -75.5, -75.0, -74.5, -74.0, -73.5,
              -73.0,
              ...
               73.0,  73.5,  74.0,  74.5,  75.0,  75.5,  76.0,  76.5,  77.0,
               77.5],
             dtype='float64', length=311)

In [34]:
ww3.__dir__()

['erddapurl',
 'datasetid',
 'protocol',
 'erddapauth',
 '_ERDDAP_Dataset__metadata',
 'resultVariables',
 'constraints',
 'serverSideFunctions',
 '_ERDDAP_Griddap__dimensions',
 '_ERDDAP_Griddap__positional_indexes',
 'lastRequestURL',
 '__module__',
 '__doc__',
 'DEFAULT_FILETYPE',
 '__init__',
 '__str__',
 'loadMetadata',
 'dimensions',
 'clearQuery',
 'loadDimensionValues',
 'getxArray',
 'getncDataset',
 'getDataRequestURL',
 'setSubset',
 'setSubsetI',
 '_parseResultVariablesExtendedDapQueryToValidDap',
 '_convertPositionalIndexes2DapQuery',
 '_resultVariablesWithValidDapIndexing',
 'positional_indexes',
 'xarray',
 'ncDataset',
 'BINARY_FILETYPES',
 '__simple_repr__',
 'setResultVariables',
 'addResultVariable',
 'setConstraints',
 'addConstraints',
 'addConstraint',
 '_addConstraintStr',
 '_addConstraintDict',
 'getURL',
 'getBaseURL',
 'getAttribute',
 'variables',
 'info',
 'getMetadataURL',
 'clearConstraints',
 'clearServerSideFunctions',
 'clearResultVariables',
 'getData'

In [37]:
ww3._ERDDAP_Griddap__dimensions['time']